## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
def missingness_summary(df, print_log, sort):
  '''
  Return a pandas series containing the percentage of missing data in each column.
  
  Args:
  df - pandas dataframe 

  print_log (boolean) - if True, print out all of the columns in the dataframe, 
    one to a line, followed by the percentage of total rows that have a 
    null value for that column.
    
  sort (string) - specify if the output should be sorted in ascending order, 
    descending order, or not at all.
  '''
  if print_log == True:
    series = pd.Series(df.isnull().sum()*100/df.isnull().count())
    if sort == 'Ascending':
      series = series.sort_values(ascending = True)
    elif sort == 'Descending':
      series = series.sort_values(ascending=False)
    else:
      series
    return series

In [4]:
missingness_summary(wine, True, 'Descending')

quality                 0.0
alcohol                 0.0
sulphates               0.0
pH                      0.0
density                 0.0
total.sulfur.dioxide    0.0
free.sulfur.dioxide     0.0
chlorides               0.0
residual.sugar          0.0
citric.acid             0.0
volatile.acidity        0.0
fixed.acidity           0.0
dtype: float64

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [5]:
# answer below
wine['quality'] = np.where((wine['quality'] >= 6), 1, 0)

Create a train test split with 20% of the data in the test subsample.

In [6]:
# answer below:
X = wine.drop('quality', axis=1)
y = wine['quality']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)

Scale only the independent variables using the minmax scaler.

In [7]:
# answer below:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [8]:
from sklearn.neighbors import KNeighborsClassifier
neighbors = KNeighborsClassifier(n_neighbors=5)
neighbors.fit(X_train, y_train)
print(neighbors.score(X_train, y_train))
print(neighbors.score(X_test, y_test))

0.7513682564503519
0.66875


In [9]:
# answer below:
neighbors = KNeighborsClassifier(n_neighbors=5)
neighbors.fit(X_scaled_train, y_train)
print(neighbors.score(X_scaled_train, y_train))
print(neighbors.score(X_scaled_test, y_test))

0.8092259577795152
0.765625


<big> Using the scaled data increased the accuracy scores for both test and train data. Both models are overfit.

When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [20]:
from math import sqrt

def weights_inv_sqrt(dist):
  with np.errstate(divide='ignore'):
    dist = 1. / np.sqrt(dist)
  inf_mask = np.isinf(dist)
  inf_row = np.any(inf_mask, axis=1)
  dist[inf_row] = inf_mask[inf_row]
  return dist

In [21]:
neighbors = KNeighborsClassifier(n_neighbors=5, weights=weights_inv_sqrt)
neighbors.fit(X_scaled_train, y_train)
print(neighbors.score(X_scaled_train, y_train))
print(neighbors.score(X_scaled_test, y_test))


1.0
0.803125
